In [ ]:
!pip install torchtune peft bitsandbytes transformers auto-gptq optimum torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.7/910.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
 

In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import login

login(token="hf_token_here")  # Replace with your Hugging Face token


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [{"role": "user", "content": "Who are you?"},]

pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct")

pipe(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta'}]}]

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:")
else:
    device = torch.device("cpu")
    print("Using CPU.")

Using GPU:


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct").to(device)

In [ ]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
comment = "Great content, thank you!"
prompt=f'''[INST] {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|>[INST] Great content, thank you! [/INST] 

I am a huge fan of the show "The Office" and I have been binge-watching it for weeks now. I'm a bit worried that my roommate is going to get tired of watching it with me. I don't want to be the only one who watches it. I was wondering, would it be possible to have a "no TV" rule, where I would have to leave the TV on when I go to my room to avoid being the only one who watches it? I know this might sound a bit strange, but I just can't bear the thought of being the only one who watches it.

I'm not sure if this is a good idea, but I want to


In [ ]:
intstructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

prompt = prompt_template(comment)
print(prompt)

[INST] ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
Great content, thank you! 
[/INST]


In [ ]:
# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to(device), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>[INST] ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
Great content, thank you! 
[/INST] 

[INST] ShawGPT responds with the following:
 
ShawGPT: 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–ShawGPT 
–


In [ ]:
model.train().to(device) # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model).to(device)

In [ ]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 524,288 || all params: 1,236,338,688 || trainable%: 0.0424


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# # Verify the file path
# file_path = "/content/drive/MyDrive/Finetuning/conversation_dataset.json"  # Update if necessary

# # Load the dataset
# data = load_dataset(file_path)

Mounted at /content/drive


In [ ]:
import json

file_path = "/content/drive/MyDrive/Finetuning/conversation_dataset_updated.json"
with open(file_path, 'r') as f:
    data = json.load(f)


dataset = load_dataset("json", data_files=file_path)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
split_dataset = dataset['train'].train_test_split(test_size=0.1,seed = 42)


In [ ]:
print("Train size:", len(split_dataset['train']))
print("Test size:", len(split_dataset['test']))

Train size: 63
Test size: 8


In [ ]:
# Iterate over both splits
for split_name, split_data in split_dataset.items():
    print(f"Processing {split_name} split:")
    for item in split_data:
        conversations = item["conversations"]  # Adjust key as per your JSON
        text = ""
        for pair in conversations:
            text += f"{pair['role'].capitalize()}: {pair['content']}\n"
        print(text)

Processing train split:
Doctor: Hey.
Patient: So I made the mistake of looking into my voicemail yesterday.
Doctor: Sorry to hear that.
Patient: I hadn't realized how long it had been since I really listen to it and I all those I was going to delete a whole bunch of messages. And so it made me listen to them before I could listen to anything else. So there were two messages from this guy with the ambulance company has been this whole thing where Blue Cross said, they weren't going to pay for any of the ambulance from one of those times one of the times I was in the hospital because it was like a non-emergency thing and even though so we had to write a grievance to them to say while I didn't have the option of refusing to take the ambulance. Yeah.
Doctor: This is like an ER to the hospital or something. Yeah.
Patient: And so that's been sent in and they've received it. Hopefully they'll pay for some of it at least but I had forgotten that that was sent in that we actually did that becau

In [ ]:
print(split_dataset['train'][0])

{'conversations': [{'content': 'Hey.', 'role': 'doctor'}, {'content': 'So I made the mistake of looking into my voicemail yesterday.', 'role': 'patient'}, {'content': 'Sorry to hear that.', 'role': 'doctor'}, {'content': "I hadn't realized how long it had been since I really listen to it and I all those I was going to delete a whole bunch of messages. And so it made me listen to them before I could listen to anything else. So there were two messages from this guy with the ambulance company has been this whole thing where Blue Cross said, they weren't going to pay for any of the ambulance from one of those times one of the times I was in the hospital because it was like a non-emergency thing and even though so we had to write a grievance to them to say while I didn't have the option of refusing to take the ambulance. Yeah.", 'role': 'patient'}, {'content': 'This is like an ER to the hospital or something. Yeah.', 'role': 'doctor'}, {'content': "And so that's been sent in and they've rec

In [ ]:
flat_data = []

for split_name, split_data in split_dataset.items():  # 'train' and 'test'
    for item in split_data:
        dialogue = item["conversations"]  # Adjust if your key is different
        text = ""
        for pair in dialogue:
            role = pair.get("role", "unknown").capitalize()
            content = pair.get("content", "")
            text += f"{role}: {content}\n"
        flat_data.append({"text": text.strip(), "split": split_name})


In [ ]:
from datasets import Dataset

flat_dataset = Dataset.from_list(flat_data)


In [ ]:
# from datasets import Dataset

# # Convert nested 'conversations' to flat 'text'
# flat_data = []

# for item in data['train']:
#     dialogue = item["conversations"]
#     text = ""
#     for turn in dialogue:
#         prefix = "Doctor: " if turn["role"] == "doctor" else "Patient: "
#         text += prefix + turn["content"].strip() + "\n"
#     flat_data.append({"text": text.strip()})

# # Create Hugging Face Dataset from the flattened data
# hf_data = Dataset.from_list(flat_data)


In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

# Step 1: Load tokenizer and assign a pad token
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

# Fix missing pad token issue
tokenizer.pad_token = tokenizer.eos_token  # ✅ Or use tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Step 2: Define tokenize function
def tokenize_function(examples):
    texts = []
    for conversation in examples['conversations']:
        text = ""
        for turn in conversation:
            prefix = "Doctor: " if turn["role"] == "doctor" else "Patient: "
            text += prefix + turn["content"].strip() + "\n"
        texts.append(text.strip())

    tokenizer.truncation_side = "left"
    return tokenizer(
        texts,
        return_tensors="np",
        padding=True,
        truncation=True,
        max_length=512
    )

# Step 3: Convert list to dataset if needed
dataset = Dataset.from_list(data)  # Make sure 'data' is your list of dicts

# Step 4: Tokenize
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["conversations"])


Map:   0%|          | 0/71 [00:00<?, ? examples/s]

In [ ]:
tokenized_train = split_dataset['train'].map(tokenize_function, batched=True, remove_columns=["conversations"])
tokenized_test = split_dataset['test'].map(tokenize_function, batched=True, remove_columns=["conversations"])

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [ ]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir="/content/drive/MyDrive/Finetuning/model2",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


<ipython-input-24-fb8c1a3bd41e>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ghibli1712 (ghibli1712-nirma-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,3.318500,3.245488
2,3.294100,3.212437
3,3.261200,3.191202
4,3.238600,3.174213
5,3.219700,3.161594
6,3.204300,3.151008
7,3.193400,3.143595
8,3.184200,3.138356
9,3.179200,3.135143
10,3.175000,3.133957


TrainOutput(global_step=40, training_loss=3.2268271923065184, metrics={'train_runtime': 260.0465, 'train_samples_per_second': 2.423, 'train_steps_per_second': 0.154, 'total_flos': 1884402745344000.0, 'train_loss': 3.2268271923065184, 'epoch': 10.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/FineTunedModels/model2")
tokenizer.save_pretrained("/content/drive/MyDrive/FineTunedModels/model2")

('/content/drive/MyDrive/FineTunedModels/model2/tokenizer_config.json',
 '/content/drive/MyDrive/FineTunedModels/model2/special_tokens_map.json',
 '/content/drive/MyDrive/FineTunedModels/model2/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_path = "/content/drive/MyDrive/FineTunedModels/model2"

model1 = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

pipe = pipeline("text-generation", model=model1, tokenizer=tokenizer, device=0)


Device set to use cuda:0


In [ ]:
# Prompt
prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a psycologist behave like one, Be polite and give sympathic responses <|eot_id|><|start_header_id|>patient<|end_header_id|>

Hello Doctor, I am Nancy. I am here to see you because I am going through mental breakdown and feeling bit low and depressed these days?<|eot_id|><|start_header_id|>Doctor<|end_header_id|>

"""





# Generate output
output = pipe(
    prompt,
    max_new_tokens=100,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)

# Print result
print(output[0]["generated_text"])


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a psycologist behave like one, Be polite and give sympathic responses <|eot_id|><|start_header_id|>patient<|end_header_id|>

Hello Doctor, I am Nancy. I am here to see you because I am going through mental breakdown and feeling bit low and depressed these days?<|eot_id|><|start_header_id|>Doctor<|end_header_id|>

I'm so sorry to hear that, Nancy. I'm here to listen and support you. Please, don't hesitate to share as much as you feel comfortable with. Can you tell me more about what's been going on and how you've been feeling?


In [ ]:
# Start the chat loop
print("💬 Doctor is online. Type 'exit' to stop.\n")

chat_history = ""

while True:
    patient_input = input("🧑‍⚕️ You (Patient): ")
    if patient_input.lower() in ["exit", "quit", "stop"]:
        print("👋 Chat ended.")
        break

    # Append the patient's input to the conversation
    chat_history += f"Patient: {patient_input}\nDoctor:"

    # Generate the doctor's response
    output = pipe(
        chat_history,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Extract and clean the response
    full_text = output[0]['generated_text']
    response = full_text[len(chat_history):].strip().split("\n")[0]  # Get only the next Doctor line
    print("🤖 Doctor:", response)

    # Update chat history
    chat_history += f" {response}\n"

💬 Doctor is online. Type 'exit' to stop.

🧑‍⚕️ You (Patient): Hey there this is Janki 
🤖 Doctor: Ah, nice to meet you, Janki. What brings you in today?
🧑‍⚕️ You (Patient): So these days I feel leftout among my friends, it's like I am with them and still i feel alone
🤖 Doctor: I understand. It sounds like you're feeling a sense of disconnection from your friends, even though you're physically present with them.
🧑‍⚕️ You (Patient): yeahhh
🤖 Doctor: It can be really tough to navigate those feelings, especially when it feels like you're not being truly seen or heard by them.
🧑‍⚕️ You (Patient): yess true its like some times they keep on ignoring me
🤖 Doctor: I see. So, you're feeling like you're being left out, not being fully present with your friends, even though you're showing up and participating in the conversations.
🧑‍⚕️ You (Patient): yesssss, what should I do to be included
🤖 Doctor: Well, first of all, let's talk about how you're feeling about being left out. That's a really tough

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


🤖 Doctor: Okay, so what if we found a way to make it a little more fun? What if we made it a little more social? What if we made it a little more like a big group activity?
🧑‍⚕️ You (Patient): yeah we can...but I don't know how//
🤖 Doctor: Okay, let's take it one step at a time. What if we started by just making it a little more fun? What if we just made it a little more social? What if we just made it a little more like a big group activity?
🧑‍⚕️ You (Patient): cool let's me invite them on my birthday coming this weekends I guess that would be good chance to socialize as well
🤖 Doctor: Okay, that sounds like a great idea! And I'm sure that would be a great chance for you to include them in your birthday celebration.
🧑‍⚕️ You (Patient): yeaa...Can you suggest some theme for my birthday party
🤖 Doctor: Okay, let's brainstorm some ideas. What if we started with something that's really fun and low-key? What if we started with something that's just a big group activity?
🧑‍⚕️ You (Patient):